# Testing pairwise synchronization performance between EDM method and ICP

In [1]:
%matplotlib widget

import matplotlib.pyplot as plt
import geoopt
import torch
import itertools
import torch.nn as nn
import numpy as np
import tqdm

import scipy.io as spio
import open3d as o3d
import networkx
import os

import copy

from sklearn.neighbors import NearestNeighbors

In [2]:
############################# GLOBAL VARS ##################################### 
# number of points to sample from each view
num_points_partviews = 500

In [3]:
# Fix random seeds
torch.manual_seed(43)
np.random.seed(43)


In [4]:
############################## DATA COLLECTION ##################################### 
QQt_ = []
bunny_views = []
bunny_views_full = []
bunny_views_pc = []
directory = 'data/bunny/views'
for filename in os.listdir(directory):
    bunny_view = o3d.io.read_point_cloud(os.path.join(directory,filename))
    bunny_view_np = np.asarray(bunny_view.points)
    # down-sampled version of point cloud
    bunny_view_down = bunny_view_np[np.random.choice(bunny_view_np.shape[0], num_points_partviews, replace=False)]

    bunny_view_c = bunny_view_down - np.mean(bunny_view_down, axis=0)
    # compute the orthogonal projector for bunny_view
    Q, R = np.linalg.qr(bunny_view_c)
    QQt_.append(torch.from_numpy(Q @ Q.T).float())
    
    bunny_views.append(bunny_view_down)
    bunny_views_full.append(bunny_view_np)
    bunny_views_pc.append(bunny_view)

numviews = len(bunny_views)

In [5]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(bunny_views[0])
o3d.visualization.draw_geometries([pcd])

In [6]:
# given input point cloud pc, compute the associated EDM
def pointsToDist(pc):
    # get size of target EDM
    m = pc.shape[0]

    # reconstructed PSD matrix
    G = pc @ pc.T
    diagonal = np.diag(G).reshape(m,1) # column vector of diag entries
    # corresponds to 2nd term in Eq (1) of main paper
    diagRows = np.tile(diagonal.T, (m, 1))
    # corresponds to 1st term in Eq (1) of main paper
    diagCols = np.tile(diagonal, (1, m))
    return diagRows + diagCols - 2 * G;

# Testing

Synchronizing two identical copies of one view, where one has randomly permuted points

In [80]:
# Below are the 3 cases for view1 and view2. Uncomment for different test

#### Noiseless case
###################
view1 = bunny_views[0]
P_view_np = np.random.permutation(np.eye(num_points_partviews))
P_view = torch.from_numpy(P_view_np).float()
view2 = P_view_np.T @ view1

#### Noise along manifold
###################
# view1 = bunny_views[0]
# view2 = bunny_views_full[0][np.random.choice(bunny_views_full[0].shape[0], num_points_partviews, replace=False)]
# P_view_np = np.zeros((num_points_partviews, num_points_partviews))
# ### We use nearest neighbors to construct 'best' association
# for j in range(num_points_partviews):
#     i = np.argmin(np.sum(np.abs(view1 - view2[j,:])**2,axis=-1))
#     P_view_np[i,j] = 1

#### Noise in full 3D space
###################
# view1 = bunny_views[0]
# P_view_np = np.random.permutation(np.eye(num_points_partviews))
# P_view = torch.from_numpy(P_view_np).float()
# view2 = P_view_np.T @ view1

# sigma = 0.003
# view2 += sigma*np.random.randn(num_points_partviews,3)


D_1 = torch.from_numpy(pointsToDist(view1)).float()
X_1 = torch.from_numpy(view1).float()
D_2 = torch.from_numpy(pointsToDist(view2)).float()
X_2_c = view2 - np.mean(view2, axis=0)
Q, R = np.linalg.qr(X_2_c)
QQt = torch.from_numpy(Q @ Q.T).float()


In [8]:
birkhoff = geoopt.manifolds.BirkhoffPolytope(tol=1e-8)
m = num_points_partviews

omega = D_2 > 0.004

with torch.no_grad():
    P = geoopt.ManifoldParameter((1/m)*torch.ones(m, m), manifold=birkhoff).proj_()


def closure():
    optim.zero_grad()
    # use this loss for noiseless cases
    loss = ((P.t() @ D_1 @ P) - D_2).pow(2).sum()
    # use this loss for both noise cases
    # loss = ((P.t() @ D_1 @ P) - D_2)[omega].pow(2).sum()

    ##### co-planar regularization
    # the particular assignment of P_i from global to view
    assign = P.t() @ X_1
    # centered assignments
    assign_c = assign - torch.mean(assign, axis=0)
    loss += (assign_c - QQt @ assign_c).pow(2).sum()

    loss.backward()
    
    return loss.item()

optim = geoopt.optim.RiemannianSGD([P], lr=8)

print_step = 100
for i in range(3000):
    curr_loss = optim.step(closure)
    if i % print_step == 0:
        print(curr_loss)


print('Done!')


8.100773811340332
0.2223963439464569
0.09607978165149689
0.06100216880440712
0.04453596472740173
0.03499339148402214
0.02877296321094036
0.02439989149570465
0.021158941090106964
0.018661946058273315
0.016679730266332626
0.015068350359797478
0.013732812367379665
0.012608573772013187
0.011648519895970821
0.010820649564266205
0.010098904371261597
0.009463608264923096
0.008901039138436317
0.009750407189130783
0.010712912306189537
0.01058490015566349
0.009950937703251839
0.010135426186025143
0.009719028137624264
0.009160637855529785
0.008941132575273514
0.008915835060179234
0.005484882276505232
0.006126316264271736
Done!


In [9]:
# Forces row distributions to choose the single element with max probability
# This generates the EDM guess for the assignments
P_np = P.detach().clone().numpy()

for i in range(num_points_partviews):
    k = np.argmax(P_np[:,i])
    P_np[:,i] = np.zeros(num_points_partviews)
    P_np[k,i] = 1

# Print percentage of correct points
p_EDM = 1 - (1/(2*num_points_partviews))*np.sum(np.abs(P_np - P_view_np))
print(p_EDM)

0.91


In [73]:
# Encapsulates above code. Takes view1 and view2 along with the respective synchronization,
# returns the percentage of correct synchronizations.
def edm_score(view1, view2, P_view):
    D_1 = torch.from_numpy(pointsToDist(view1)).float()
    X_1 = torch.from_numpy(view1).float()
    D_2 = torch.from_numpy(pointsToDist(view2)).float()
    X_2_c = view2 - np.mean(view2, axis=0)
    Q, R = np.linalg.qr(X_2_c)
    QQt = torch.from_numpy(Q @ Q.T).float()

    birkhoff = geoopt.manifolds.BirkhoffPolytope(tol=1e-8)
    m = view1.shape[0]

    with torch.no_grad():
        P = geoopt.ManifoldParameter((1/m)*torch.ones(m, m), manifold=birkhoff).proj_()


    def closure():
        optim.zero_grad()
        # use this loss for noiseless cases
        loss = ((P.t() @ D_1 @ P) - D_2).pow(2).sum()
        # use this loss for both noise cases
        # loss = ((P.t() @ D_1 @ P) - D_2)[omega].pow(2).sum()

        ##### co-planar regularization
        # the particular assignment of P_i from global to view
        assign = P.t() @ X_1
        # centered assignments
        assign_c = assign - torch.mean(assign, axis=0)
        loss += (assign_c - QQt @ assign_c).pow(2).sum()

        loss.backward()
        
        return loss.item()

    optim = geoopt.optim.RiemannianSGD([P], lr=8)
    print_step = 100
    for i in range(1200):
        curr_loss = optim.step(closure)
        if i % print_step == 0:
             print(curr_loss)

    P_np = P.detach().clone().numpy()

    for i in range(m):
        k = np.argmax(P_np[:,i])
        P_np[:,i] = np.zeros(m)
        P_np[k,i] = 1

    # Print percentage of correct points
    p_EDM = 1 - (1/(2*m))*np.sum(np.abs(P_np - P_view))
    return p_EDM


# ICP METHOD

In [88]:
# Code from https://github.com/ClayFlannigan/icp
def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform that maps corresponding points A to B in m spatial dimensions
    Input:
      A: Nxm numpy array of corresponding points
      B: Nxm numpy array of corresponding points
    Returns:
      T: (m+1)x(m+1) homogeneous transformation matrix that maps A on to B
      R: mxm rotation matrix
      t: mx1 translation vector
    '''

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
       Vt[m-1,:] *= -1
       R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(m+1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t


def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nxm array of points
        dst: Nxm array of points
    Output:
        distances: Euclidean distances of the nearest neighbor
        indices: dst indices of the nearest neighbor
    '''

    assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()


def icp(A, B, init_pose=None, max_iterations=2000, tolerance=1e-8):
    '''
    The Iterative Closest Point method: finds best-fit transform that maps points A on to points B
    Input:
        A: Nxm numpy array of source mD points
        B: Nxm numpy array of destination mD point
        init_pose: (m+1)x(m+1) homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation that maps A on to B
        distances: Euclidean distances (errors) of the nearest neighbor
        i: number of iterations to converge
    '''

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1,A.shape[0]))
    dst = np.ones((m+1,B.shape[0]))
    src[:m,:] = np.copy(A.T)
    dst[:m,:] = np.copy(B.T)

    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        # find the nearest neighbors between the current source and destination points
        distances, indices = nearest_neighbor(src[:m,:].T, dst[:m,:].T)

        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[:m,:].T, dst[:m,indices].T)

        # update the current source
        src = np.dot(T, src)

        # check error
        mean_error = np.mean(distances)
        if np.abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error

    # calculate final transformation
    _,R,t = best_fit_transform(A, src[:m,:].T)

    return R, t, distances, i

In [139]:
# how many distretizations to make for graph
refinement = 1
# furthest rotation to consider
total_dist = np.pi/2
# different values for rotation displacement
vals = np.zeros(refinement)
# percentage correct by ICP on respective rotation
p_correct = np.zeros(refinement)

for k in range(refinement):
    # amount of translation displacement. for now, we assume no displacement
    sigma_1 = 0.0086
    # angle of displacement for rotation
    u = 0.15 * total_dist
    vals[k] = u / np.pi
    # Generate random rotation. We do this by determining what vector (1,0,0) rotates to
    x = np.zeros((3,1))
    x[0] = 1
    y = np.zeros((3,1))
    y[0] = np.cos(u)
    y[1] = np.sin(u)

    R_init = np.eye(3) + y @ x.T - x @ y.T + (1/(1 + np.inner(x,y)))*np.linalg.matrix_power(y@x.T - x@y.T,2)
    # make sure not a reflection
    if np.linalg.det(R_init) < 0:
        R_init = -1*R_init

    ICP_1 = view1
    mean = np.mean(view2,axis=0)
    ICP_2 = (view2 - mean) @ R_init.T + mean + sigma_1 * np.random.randn(ICP_1.shape[0],3)

    R_icp, t_icp, dist, ite = icp(ICP_1, ICP_2, max_iterations=2000, tolerance=1e-8)

    # ######### determine the percentage of correct associations via nearest neighbor
    mean =  np.mean(ICP_1, axis=0)
    ICP_1_fixed = (ICP_1 - mean) @ R_icp.T
    # ICP_1_fixed = ICP_1_fixed + mean - t_icp
    ICP_2_fixed = ICP_2 - np.mean(ICP_2, axis=0)

    P_icp = np.zeros((num_points_partviews, num_points_partviews))
    for j in range(num_points_partviews):
        i = np.argmin(np.sum(np.abs(ICP_1_fixed - ICP_2_fixed[j,:])**2,axis=-1))
        P_icp[i,j] = 1

    p_correct[k] = 1 - (1/(2*num_points_partviews))*np.sum(np.abs(P_icp - P_view_np))
print(p_correct[0])

0.124


In [104]:
figure = plt.figure()
f = figure.add_subplot(111, projection='3d')
f.scatter(ICP_1_fixed[:,0], ICP_1_fixed[:,1], ICP_1_fixed[:,2], c='r', s = 3)
f.scatter(ICP_2_fixed[:,0], ICP_2_fixed[:,1], ICP_2_fixed[:,2], c='b', s = 3)

f.set_xlabel('X')
f.set_ylabel('Y')
f.set_zlabel('Z')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fast Global Registration (FGR)

Implementation follows http://www.open3d.org/docs/release/tutorial/Advanced/global_registration.html

In [6]:
# Method for preparing data for FGR. Additional parameter P for possible permutation of points
def preprocess_point_cloud(pcd, voxel_size, *transformation):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)
    # If specified, transform the downsampled point cloud
    if len(transformation) == 1: # just given sigma
        sigma = transformation[0]
        pcd_np = np.asarray(pcd_down.points)
        m = pcd_np.shape[0]
        pcd_down.points = o3d.utility.Vector3dVector(pcd_np + sigma*np.random.randn(m,3))
    if len(transformation) == 4:
        P = transformation[0]
        R = transformation[1]
        t = transformation[2]
        sigma = transformation[3]
        
        pcd_np = np.asarray(pcd_down.points)
        m = pcd_np.shape[0]
        pcd_down.points = o3d.utility.Vector3dVector(P.T @ pcd_np + sigma*np.random.randn(m,3))

        pcd_down.rotate(R, np.mean(pcd_np, axis=0))

        pcd_down.translate(t)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [7]:
# Method for executing FGR
def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.registration.registration_fast_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

In [8]:
# Method for drawing the converged result
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
                                    #   zoom=0.4559,
                                    #   front=[0.6452, -0.3036, -0.7011],
                                    #   lookat=[1.9892, 2.0208, 1.8945],
                                    #   up=[-0.2779, -0.9482 ,0.1556])

In [17]:
view1_down, view1_fpfh = preprocess_point_cloud(bunny_views_pc[0], voxel_size, sigma)
print(view1_fpfh.data.shape)


:: Downsample with a voxel size 0.007.
:: Estimate normal with search radius 0.014.
:: Compute FPFH feature with search radius 0.035.
(33, 767)


In [10]:
# Single instance of below comparison graph generator

# Note we also set voxel size to be the max sigma
# bunny voxel size
# voxel_size = 0.0086
# dragon voxel size
# voxel_size = 0.00947
# buddah voxel size
# voxel_size = 0.0077
# armadillo voxel size
voxel_size = 0.00697

sigma = voxel_size / 4

view1_down, view1_fpfh = preprocess_point_cloud(bunny_views_pc[0], voxel_size, sigma)
m = np.asarray(view1_down.points).shape[0]
print(m)
# Size gotten empirically from running above command
P_view = np.random.permutation(np.eye(m))
x = np.zeros((3,1))
x[0] = 1
y = np.zeros((3,1))
y[1] = -1
y[2] = -0.9
y = (1/np.linalg.norm(y)) * y
R = np.eye(3) + y @ x.T - x @ y.T + (1/(1 + np.inner(x,y)))*np.linalg.matrix_power(y@x.T - x@y.T,2)
t = 0.1*np.ones((3,1))


view2_down, view2_fpfh = preprocess_point_cloud(bunny_views_pc[0],voxel_size, P_view, R, t, sigma)
print('EDM score w/ noise %d' % (sigma))
print(edm_score(np.asarray(view1_down.points), np.asarray(view2_down.points), P_view))
# start = time.time()
result_fast = execute_fast_global_registration(view1_down, view2_down, view1_fpfh, view2_fpfh, voxel_size)
# print("Fast global registration took %.3f sec.\n" % (time.time() - start))
fgr_corr = np.asarray(result_fast.correspondence_set)
total = 0
for i in range(fgr_corr.shape[0]):
    # check how many assignments truly are valid
    total += P_view[fgr_corr[i,0], fgr_corr[i,1]]
# NOTE: we always know size=509. Otherwise P_view would return
# an error when multiplied through pc

print('FGR score w/ noise %d' % (sigma))
print(total / m)

:: Downsample with a voxel size 0.007.
:: Estimate normal with search radius 0.014.
:: Compute FPFH feature with search radius 0.035.
767
:: Downsample with a voxel size 0.007.
:: Estimate normal with search radius 0.014.
:: Compute FPFH feature with search radius 0.035.
EDM score w/ noise 0


NameError: name 'edm_score' is not defined

In [25]:
draw_registration_result(view1_down, view2_down, result_fast.transformation)

In [32]:
# Create graphs of performance vs. noise for EDM vs. FGR

# Note we also set voxel size to be the max sigma
voxel_size = 0.0086
refinement = 10

edm_scores = np.zeros(refinement)
fgr_scores = np.zeros(refinement)
fgr_scores_relative = np.zeros(refinement)

view1_down, view1_fpfh = preprocess_point_cloud(bunny_views_pc[0], voxel_size)
# Size gotten empirically from running above command
P_view = np.random.permutation(np.eye(509))
x = np.zeros((3,1))
x[0] = 1
y = np.zeros((3,1))
y[1] = -1
y[2] = -0.9
y = (1/np.linalg.norm(y)) * y
R = np.eye(3) + y @ x.T - x @ y.T + (1/(1 + np.inner(x,y)))*np.linalg.matrix_power(y@x.T - x@y.T,2)
t = 0.1*np.ones((3,1))

for z in range(refinement):
    sigma = z/(refinement-1)*voxel_size
    view2_down, view2_fpfh = preprocess_point_cloud(bunny_views_pc[0],voxel_size, P_view, R, t, sigma)
    edm_scores[z] = edm_score(np.asarray(view1_down.points), np.asarray(view2_down.points), P_view)
    print('EDM score w/ noise %d' % (sigma))
    print(edm_scores[z])
    # start = time.time()
    result_fast = execute_fast_global_registration(view1_down, view2_down, view1_fpfh, view2_fpfh, voxel_size)
    # print("Fast global registration took %.3f sec.\n" % (time.time() - start))
    fgr_corr = np.asarray(result_fast.correspondence_set)
    total = 0
    for i in range(fgr_corr.shape[0]):
        # check how many assignments truly are valid
        total += P_view[fgr_corr[i,0], fgr_corr[i,1]]
    # NOTE: we always know size=509. Otherwise P_view would return
    # an error when multiplied through pc
    fgr_scores[z] = total / 509
    fgr_scores_relative[z] = total / (fgr_corr.shape[0])

    print('FGR score w/ noise %d' % (sigma))
    print(fgr_scores[z])


:: Downsample with a voxel size 0.015.
:: Estimate normal with search radius 0.030.
:: Compute FPFH feature with search radius 0.075.
223
:: Downsample with a voxel size 0.015.
:: Estimate normal with search radius 0.030.
:: Compute FPFH feature with search radius 0.075.
2.8642725944519043
0.08610578626394272
0.03545970097184181
0.022625071927905083
0.016623614355921745
0.01311052218079567
0.010793756693601608
0.009147979319095612
0.007917625829577446
0.00696287676692009
0.006200648378580809
0.005578284617513418
0.005060834810137749


KeyboardInterrupt: 

In [157]:
edm_scores_fixed = np.zeros(refinement) #accidently scaled by 500, not 509
sigmas = np.zeros(refinement)

for i in range(refinement):
    edm_scores_fixed[i] = 1 - (500/509)*(1 - edm_scores[i])
    sigmas[i] = i/(refinement-1)*voxel_size

figure = plt.figure()
plt.plot(sigmas, fgr_scores)
plt.plot(sigmas, edm_scores_fixed)
plt.legend(['FGR', 'EDM Method'])
plt.xlabel('$\sigma$')
plt.ylabel('% correct associations')
# plt.title('Synchronization Performance of ICP by Rotational Separation')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [140]:
sigmas = np.array([0., 0.00095556, 0.00191111, 0.00286667, 0.00382222, 0.00477778, 0.00573333, 0.00668889, 0.00764444, 0.0086])
edm_scores = np.array([0.98428291, 0.94695481, 0.76227898, 0.55009823, 0.33398821, 0.28683694, 0.2043222, 0.15717092, 0.00196464, 0.00196464])
fgr_scores = np.array([1., 0.97445972, 0.66601179, 0.21611002, 0., 0.00196464, 0., 0., 0., 0.])
icp_scores = np.array([0.982, 0.866, 0.624, 0.45799999999999996, 0.348, 0.256, 0.19399999999999995, 0.16799999999999993, 0.14, 0.124])

In [142]:
figure99 = plt.figure()
plt.plot(sigmas, fgr_scores, linewidth=3)
plt.plot(sigmas, icp_scores, linewidth=3)
plt.plot(sigmas, edm_scores, linewidth=3)
plt.legend(['FGR', 'ICP', 'EDM Method (ours)'])
plt.xlabel('$\sigma$ of random noise')
plt.ylabel('% correct associations')
font = {'family' : 'normal',
        'size'   : 12}

plt.rc('font', **font)
# plt.title('Synchronization Performance of ICP by Rotational Separation')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Comparison

In [127]:
figure18 = plt.figure()
plt.plot(vals, p_correct, linewidth=3)
plt.plot(vals, 0.89*np.ones(refinement), linewidth=3)
plt.legend(['ICP', 'EDM Method'])
plt.xlabel('$\pi$ radians of rotation')
plt.ylabel('% correct associations')
# plt.title('Synchronization Performance of ICP by Rotational Separation')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …